In [1]:
import ee
import geopandas as gpd
import geemap
import pandas as pd
from shapely.geometry import mapping, Polygon, MultiPolygon, Point, LineString

In [2]:
ee.Authenticate()

True

In [4]:
ee.Initialize(project='agri-gis-467304')
print('The initialization process has been completed successfully.')

The initialization process has been completed successfully.


In [8]:
def Shp_to_ee_fc(shp_path):
    """
    Convert a GeoPandas GeoDataFrame to an Earth Engine FeatureCollection.
    Strips Z coordinates from all geometry types.
    """
    gdf = gpd.read_file(shp_path)

    def to_2d(geom):
        if geom is None or geom.is_empty:
            return None
        # Point
        if geom.geom_type == 'Point':
            x, y = geom.coords[0][:2]
            return Point(x, y)
        # LineString
        elif geom.geom_type == 'LineString':
            coords = [(c[0], c[1]) for c in geom.coords]
            return LineString(coords)
        # Polygon
        elif geom.geom_type == 'Polygon':
            coords = [(c[0], c[1]) for c in geom.exterior.coords]
            return Polygon(coords)
        # MultiPolygon
        elif geom.geom_type == 'MultiPolygon':
            polys = []
            for poly in geom.geoms:
                coords = [(c[0], c[1]) for c in poly.exterior.coords]
                polys.append(Polygon(coords))
            return MultiPolygon(polys)
        else:
            return None

    features = []
    for _, row in gdf.iterrows():
        geom2d = to_2d(row.geometry)
        if geom2d is None:
            continue

        # Drop geometry column from properties
        props = row.drop('geometry').to_dict()
        
        if geom2d.geom_type == 'MultiPolygon':
            for poly in geom2d.geoms:
                features.append(ee.Feature(ee.Geometry(mapping(poly)), props))
        else:
            features.append(ee.Feature(ee.Geometry(mapping(geom2d)), props))

    return ee.FeatureCollection(features)


In [9]:
shp_path = r"D:\MPSEDC_Internship\Row_Data\Final_Points\Point.shp"
ee_fc = Shp_to_ee_fc(shp_path)

In [10]:
ee_fc